环境安装

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

根据任务需求调整参数组合

In [ ]:
from unsloth import FastLanguageModel # 导入Unsloth优化版模型加载类
import torch #PyTorch深度学习框架
max_seq_length = 1024 # 模型支持的最大输入序列长度
dtype = None   # 自动选择计算数据类型（通常为bfloat16或float16）
load_in_4bit = True
# 加载模型与分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Deepseek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length, # 设定模型处理的最大序列长度
    load_in_4bit = True,  # 数据类型自动选择（None表示由库决定）
    dtype=dtype, # 以4-bit量化加载模型，显著减少显存占用
    #llm_int8_enable_fp32_cpu_offload=True # 允许将部分计算卸载到CPU的FP32精度（节省GPU显存）
)

In [ ]:
prompt_style = """下面是一条描述任务的指令，与提供进一步上下文的输入配对。
请求对适当完成请求的响应的答案。
在回答之前，仔细思考问题，并创建一个循序渐进的思路链，以确保逻辑清晰、准确的回答.

### Instruction:
您是一位劳动能力鉴定专家，在工伤认定、工伤鉴定，劳动能力鉴定方面拥有先进的知识。
请回答以下劳动能力鉴定问题。

### Question:
{}

### Response:
<think>{}"""

可扩展参数（若需多样性）：
temperature=0.7：调节随机性。
do_sample=True：启用采样策略。
top_p=0.9：核采样

In [ ]:

question = "2024年10月25日8时40分左右，该单位职工刘丰硕在因工外出期间，在我家阳光上城小区南侧采集原有电杆及新建电杆坐标点信息时，不慎踩空摔倒在沟渠中伤及右膝。医院诊断为：半月板损伤（右）。请看一下鉴定等级和以及的工伤保险条例"

FastLanguageModel.for_inference(model)  #  # 启用Unsloth的2倍加速推理优化
# 使用预设提示模板格式化输入（假设 prompt_style = "### Question:\n{}\n\n### Response:\n{}"）


inputs = tokenizer([prompt_style.format(question, "")],
                   return_tensors="pt" # 返回PyTorch张量
                  ).to("cuda")  # 将输入数据移至GPU
outputs = model.generate(
    input_ids=inputs.input_ids, # 编码后的输入ID
    attention_mask=inputs.attention_mask,  # 标识有效token位置
    max_new_tokens=1200,  # 最大生成token数（控制回答长度）
    use_cache=True, # 启用KV缓存加速生成
)
response = tokenizer.batch_decode(outputs)  # 将token ID解码为文本
print(response[0].split("### Response:")[1])  # 提取"### Response:"后的内容展示

In [ ]:
from datasets import load_dataset  #引入huggingface dataset
dataset = load_dataset("liyuaneli/liyuanlei",split ="train[0:10]",trust_remote_code=True) #加载dataset
print(dataset.column_names)

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # 获取分词器的终止符（如</s>）

def formatting_prompts_func(examples):
    inputs = examples["Question"]   # 问题列表（如["如何泡茶？", ...]）
    cots = examples["Complex_CoT"]  # 思维链列表（如["步骤1：烧水...", ...]）
    outputs = examples["Response"]  # 答案列表（如["首先，准备茶叶...", ...]）
    texts = []
     # 遍历每个样本，拼接问题、CoT、答案和EOS_TOKEN
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,       # 返回包含格式化文本的字典（键需与SFTTrainer的dataset_text_field匹配）
    }

In [ ]:
train_prompt_style = """下面是一条描述任务的指令，与提供进一步上下文的输入配对。
请求对适当完成请求的响应的答案。
在回答之前，仔细思考问题，并创建一个循序渐进的思路链，以确保逻辑清晰、准确的回答.

### Instruction:
您是一位劳动能力鉴定专家，在工伤认定、工伤鉴定，劳动能力鉴定方面拥有先进的知识。
请回答以下劳动能力鉴定问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset["text"][0]

In [ ]:
FastLanguageModel.for_training(model)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,  # 基础模型（如LLaMA、Mistral等预训练模型）
    r=16,   # LoRA的秩（rank），控制低秩矩阵的维度
    target_modules=[ # 应用LoRA的目标模块列表
        "q_proj", # 查询（Query）投影层
        "k_proj", # 键（Key）投影层
        "v_proj", # 值（Value）投影层
        "o_proj", # 输出（Output）投影层
        "gate_proj",  # 门控投影层（FFN模块）
        "up_proj", # 上投影层（FFN模块）
        "down_proj", # 下投影层（FFN模块）
    ],
    lora_alpha=16, # LoRA缩放因子（控制适配器对原始权重的影响）
    lora_dropout=0, # LoRA层的Dropout率（防止过拟合）
    bias="none",     # 是否在LoRA层中添加偏置（"none"表示不添加）
    use_gradient_checkpointing="unsloth",  # 使用梯度检查点技术（节省显存）
    random_state=3407, # 随机种子（确保实验可复现）
    use_rslora=False, # 是否使用Reversed LoRA（一种变体，此处未启用）
    loftq_config=None, # LoRA量化配置（此处未启用）
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    # 基础模型与数据配置
    model=model, # 待微调的预训练模型（如加载的LLaMA）
    tokenizer=tokenizer, # 分词器
    train_dataset=dataset, # 训练数据集（格式需包含"text"字段）
    dataset_text_field="text", # 指定数据集中文本内容对应的字段名
    max_seq_length=max_seq_length,  # 输入序列的最大长度（如2048）
    dataset_num_proc=2, # 数据预处理进程数（并行加速）
    packing = False, # 是否将多个样本打包为连续序列（节省显存，但可能影响效果）
     # 训练参数配置（通过TrainingArguments封装）
    args=TrainingArguments(
         # 批量与梯度配置
        per_device_train_batch_size=2, # 每个GPU的批量大小
        gradient_accumulation_steps=4, # 梯度累积步数（等效批量=2*4=8）
        # Use num_train_epochs = 1, # 训练轮次
        # 训练周期与步数
        warmup_steps=5, #学习率预热步数（避免初始震荡）
        max_steps=60, # 最大训练步数（优先级高于epochs）
         # 学习率与优化器
        learning_rate=2e-4, # 初始学习率（常用范围：1e-5~3e-4）
        fp16=not is_bfloat16_supported(),  # 启用FP16混合精度（若不支持BF16）
        bf16=is_bfloat16_supported(),  # 优先启用BF16（若硬件支持，如A100）
        logging_steps=1, # 每1步记录一次日志
        optim="adamw_8bit", # 使用8-bit AdamW优化器（节省显存）
        weight_decay=0.01, # 权重衰减（防止过拟合）
        lr_scheduler_type="linear", # 学习率调度策略（线性衰减）
        seed=3407, # 固定随机种子（确保实验可复现）
        output_dir="outputs", # 模型与日志输出目录
        report_to ="none", # 不向外部平台（如W&B）报告结果
    ),
)

In [ ]:
trainer_stats = trainer.train();

In [ ]:
if True: model.save_pretrained_gguf("model", tokenizer,)
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

In [ ]:
from google.colab import userdata
HUGGINGFACE_TOKEN = userdata.get("HUGGINGFACE_TOKEN");

In [ ]:
from huggingface_hub import create_repo
print(HUGGINGFACE_TOKEN)
create_repo("liyuaneli/r1-test-train-model",token=HUGGINGFACE_TOKEN,exist_ok=True)

In [ ]:
model.push_to_hub_gguf("liyuaneli/r1-test-train-model",tokenizer,token=HUGGINGFACE_TOKEN)